# TAL - Lab05 word2vec

## 1. Test et évalutation d'un modèle entraîné sur Google News

Téléchargement du moèdle `word2vec`

In [2]:
import gensim.downloader as api
from gensim.models import KeyedVectors, Word2Vec
from gensim.models.word2vec import Text8Corpus
from gensim.test.utils import datapath

/home/ducky/.conda/envs/tal/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [13]:
# run this cell if you don't have the model saved locally
w2v_model = api.load("word2vec-google-news-300")

In [3]:
# run this cell if you already have the model saved locally
path_to_file = '~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz'
w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)

> b. Quelle place mémoire occupe le processus du notebook une fois les vecteurs de mots chargés ?  

Le processus utilise **~4GB** de mémoire.

> c. Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés?  

La dimension est de **300**

In [4]:
w2v_vectors.vector_size

300

> d. Quelle est la taille du vocabulaire du modèle?  

La taille du vocabulaire est du **3'000'000**

In [5]:
words = ["dog", "cat", "teacher", "engineer", "switzerland", "t-shirt", "xylography"]
w2v_vectors.vectors.shape

for word in words:
    print("is {} in the vocabulary? {}".format(word, w2v_vectors.has_index_for(word)))

is dog in the vocabulary? True
is cat in the vocabulary? True
is teacher in the vocabulary? True
is engineer in the vocabulary? True
is switzerland in the vocabulary? True
is t-shirt in the vocabulary? False
is xylography in the vocabulary? False


> e. Quelle est la distance entre les mots rabbit et carrot?  

La distance entre `rabit` et `carrot` est **0.636**. Cette distance et calculé grace à la **similarité cosinus**.

In [6]:
w2v_vectors.distance("rabbit", "carrot")

0.6369356513023376

> f. Considérez 5-10 paires de mots, certains proches sémantiquement, d’autres non.  Pour chaque paire, calculez la distance entre les deux mots.  Veuillez indiquer si les distances obtenues correspondent à vos intuitions sur la proximité des sens des mots.  

Correspond a notre intuitions:  
* ("friend", "family") - Oui  
* ("cat", "kitten") - Oui  
* ("sword", "ball") - Oui/Non (on pensait qu'ils seraient plus loin)  
* ("shirt", "jeans") - Oui  
* ("bed", "plane") - Oui  
* ("old", "cold") - Oui  
* ("tea", "eejit") - Oui  
* ("computer", "programmer") - Oui/Non (on pensait qu'ils seraient plus proche)  
* ("nurse", "doctor") - Oui  

In [7]:
word_pairs = [
    ("friend", "family"),
    ("cat", "kitten"),
    ("sword", "ball"),
    ("shirt", "jeans"),
    ("bed", "plane"),
    ("old", "cold"),
    ("tea", "eejit"),
    ("computer", "programmer"),
    ("nurse", "doctor"),
]

for word_pair in word_pairs:
    print("distance between '{}' & '{}' is {:.4f}".format(
        word_pair[0], 
        word_pair[1], 
        w2v_vectors.distance(word_pair[0], word_pair[1])
    ))

distance between 'friend' & 'family' is 0.5108
distance between 'cat' & 'kitten' is 0.2535
distance between 'sword' & 'ball' is 0.7112
distance between 'shirt' & 'jeans' is 0.4398
distance between 'bed' & 'plane' is 0.8260
distance between 'old' & 'cold' is 0.9185
distance between 'tea' & 'eejit' is 0.9207
distance between 'computer' & 'programmer' is 0.5749
distance between 'nurse' & 'doctor' is 0.3680


> g. Pouvez-vous trouver des mots de sens opposés mais qui sont proches dans l’espace vectoriel?  

`buy` et `sell`

> Comment expliquez vous cela, et est-ce que c’est selon vous une qualité ou un défaut du modèle word2vec?  

Se sont certes des mots de sens opposés mais ils ont un fort lien entre eux (C-à-d l'un ne peu pas exister sans l'autre).
C'est clairement une qualité car cela montre que le modèle prend aussi en compte les liens cachés entre les mots.

In [8]:
w2v_vectors.distance("buy", "sell")

0.1691538691520691

> h. Trouvez un mot (anglais) qui possède plusieurs sens différents, et pour chaque sens trouvez un mot sémantiquement proche. (Par exemple, en français, avocat—procureur et avocat—banane.) Comparez les différentes distances. Quel est le défaut du modèle word2vec que vous observez?

C'est que ces pairs de mots on des sens très proche, mais ils sont très eloignés selon le modèle.

In [9]:
# bow - arrow
# bow - tie
# bow - boat
homograph_pairs = [
    ("bow", "arrow"),
    ("bow", "tie"),
    ("bow", "boat")
]

for homograph_pair in homograph_pairs:
    print("distance between '{}' & '{}' is {:.4f}".format(
        homograph_pair[0],
        homograph_pair[1],
        w2v_vectors.distance(homograph_pair[0], homograph_pair[1])
    ))

distance between 'bow' & 'arrow' is 0.6049
distance between 'bow' & 'tie' is 0.7186
distance between 'bow' & 'boat' is 0.7089


> i. Calculez le score du modèle word2vec sur les données **WordSimilarity-353** et expliquer en 1-2 phrases comment ce score est calculé.

Il calcul 3 score different:
1. le coefficiant de Pearson 
2. la correlation de Spearman
3. OOV-ratio: Le ratio de pair qui a des mots inconnus

In [45]:
gnews_wordsim = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

In [46]:
# Attention bloc lent !!!!!
gnews_qwords = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))

> i. En vous aidant de la documentation,calculez le score dumodèle word2vec sur les données **questions-words.txt**. Expliquez en 1-2 phrases comment ce score est calculé (donc ce qu’il mesure)

La fonction renvoie un tupple, la première valeur est le ratio de question où le modèle a répondu correctement, la deuxième valeur est les données utilisé pour le score.

Le modèle calcul sa réponse en faisant: Athens - Greece + Bangkok et il regarde si Thailand est le mot le plus proche du résultat. Si oui la réponse est considérée comme correct.

In [49]:
print("Results:")
print("--------")

print()

print("Wordsim353:")
print(gnews_wordsim)

print()

print("Questions Words:")
print(gnews_qwords[0])

Results:
--------

Wordsim353:
((0.6238773466616107, 1.7963237724171284e-39), SpearmanrResult(correlation=0.6589215888009288, pvalue=2.5346056459149263e-45), 0.0)

questions-words:
0.7401448525607863


## 2. Entraîner deux nouveaux modèles word2vec à partir de nouveaux corpus

> a. En utilisant `gensim.downloader`, récupérez le corpus qui contient les 10⁸ premiers caractères de Wikipédia (en anglais) avec la commande : `corpus = gensim.downloader.load('text8')`. Combien de phrases et de mots (tokens) possède ce corpus?

In [41]:
text8_corpus = api.load('text8')

In [73]:
# Nombre de phrases : 1701
print("Number of sentences", api.info('text8')['num_records'])

Number of sentences 1701


In [74]:
# Nombre de mots : 17005207
print("Number of words: ", sum(len(i) for i in text8_corpus))

Number of words:  17005207


> b. Entraînez un nouveau modèle word2vec sur ce nouveau corpus. Si nécessaire, procédez progressivement, en commençant par **1%** du corpus, puis **10%**, pour contrôler le temps nécessaire.
> - Indiquez la dimension choisie pour le *embedding* de ce nouveau modèle
> - Combien de temps prend l’entraînement sur le corpus total?
> - Quelle est la taille (en Mo) du modèle word2vec résultant ?

Temps = 2 minutes  
Taille du modele = 57 Mo

In [55]:
# run this cell if you don't have the model saved locally
text8_model = Word2Vec(corpus, vector_size=300)
text8_model.save("text8.model")

In [16]:
# run this cell if you already have the model saved locally
text8_model = KeyedVectors.load("text8.model", mmap='r')

> c. Mesurez la qualité de ce modèle comme dans la partie 1, points i et j. Ce modèle est-il meilleur que celui entraîné sur Google News? Quelle serait la raison de la différence?

Ce modèle n'est pas meilleur que celui qui est entraîné sur Google News. La raison de cette différence provient de la taille des datasets. Dans ce cas, le dataset possède 71k tokens tandis que celui de Google news en a 3M.

In [56]:
text8_wv = text8_model.wv

In [57]:
text8_wordsim = text8_wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

In [58]:
text8_qwords = text8_wv.evaluate_word_analogies(datapath('questions-words.txt'))

In [72]:
print("Results:")
print("--------")

print()

print("Wordsim353:")
print("Google News:", gnews_wordsim)
print("Text8:", text8_wordsim)

print()

print("Questions Words:")
print("Google News:", gnews_qwords[0])
print("Text8:", text8_qwords[0])

Results:
--------

Wordsim353:
Google News: ((0.6238773466616107, 1.7963237724171284e-39), SpearmanrResult(correlation=0.6589215888009288, pvalue=2.5346056459149263e-45), 0.0)
Text8: ((0.6105890642318692, 3.072339335590552e-37), SpearmanrResult(correlation=0.6246376132773954, pvalue=2.2456458997037836e-39), 0.56657223796034)

Questions Words:
Google News: 0.7393798449612403
Text8: 0.26256513699781475


> d. Téléchargez maintenant le corpus quatre fois plus grandc onstitué de la concaténation du corpus `text8` et des dépêches économiques de Reuters (413 Mo) fourni en ligne par l’enseignant et appelé `wikipedia_augmented.dat`. Entraînez un nouveau modèle word2vecsur ce corpus, en précisant la dimension du plongement (embedding). 
> - Utilisez la classe `Text8Corpus()` pourcharger le corpus et faire la tokenization et la segmentation en phrases
> - Combien de temps prend l’entraînement?
> - Quelle est la taille (en Mo) du modèle word2vec résultant?

Temps = 5 minutes  
Taille = 100M

In [48]:
# run this cell if you don't have the model saved locally
wiki_augmented_model = Word2Vec(Text8Corpus('wikipedia_augmented.dat'))
wiki_augmented_model.save("wiki_augmented.model")

In [29]:
# run this cell if you already have the model saved locally
wiki_augmented_model = Word2Vec.load("wiki_augmented.model", mmap='r')

> e. Testez ce modèle comme en 1.i et 1.j.  Est-il meilleur que le précédent? Pour quelle raison?

Il est meilleur que notre modèle sur text-8. Néanmoins, il est moins bon que celui de Google News. La raison est la même que précédemment. Il y a une grande différence de taille (~120K vs 3M).

In [50]:
wiki_augmented_wv = wiki_augmented_model.wv

In [51]:
wiki_wordsim = wiki_augmented_wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

In [52]:
wiki_qwords = wiki_augmented_wv.evaluate_word_analogies(datapath('questions-words.txt'))

In [71]:
print("Results:")
print("--------")

print()

print("Wordsim353:")
print("Google News:", gnews_wordsim)
print("Text8:", text8_wordsim)
print("Wikipedia augmented:", wiki_wordsim)

print()

print("Questions Words:")
print("Google News:", gnews_qwords[0])
print("Text8:", text8_qwords[0])
print("Wikipedia augmented:", wiki_qwords[0])

Results:
--------

Wordsim353:
Google News: ((0.6238773466616107, 1.7963237724171284e-39), SpearmanrResult(correlation=0.6589215888009288, pvalue=2.5346056459149263e-45), 0.0)
Text8: ((0.6105890642318692, 3.072339335590552e-37), SpearmanrResult(correlation=0.6246376132773954, pvalue=2.2456458997037836e-39), 0.56657223796034)
Wikipedia augmented: ((0.5050250395258843, 3.0228907961391866e-24), SpearmanrResult(correlation=0.512229711290635, pvalue=5.258117495412604e-25), 0.0)

Questions Words:
Google News: 0.7393798449612403
Text8: 0.26256513699781475
Wikipedia augmented: 0.34995654063450676


> f. Créez un nouveau fichier de test en augmentant `questions-words.txt` avec environ 20 items de test. Par exemple, à partir de `{(eye, see), (ear, listen), (foot, walk)}` on peut construire 6 items de test. Testez les trois modèles sur ces nouvelles données, puis commentez vos résultats

Il n'y a eu aucun changements. Ce qui est normal étant donnée que nous avons seulement ajouté 20 items de test ce qui n'est pas grand chose étant donné qu'il y a déjà ~20'000 qestions.

In [61]:
gnews_qwords = w2v_vectors.evaluate_word_analogies('questions-words-custom.txt')

In [62]:
text8_qwords = text8_wv.evaluate_word_analogies('questions-words-custom.txt')

In [63]:
wiki_qwords = wiki_augmented_wv.evaluate_word_analogies('questions-words-custom.txt')

In [64]:
print("Results:")
print("--------")
print("Google News:", gnews_qwords[0])
print("Text8:", text8_qwords[0])
print("Wikipedia augmented:", wiki_qwords[0])

Results:
--------
Google News: 0.7393798449612403
Text8: 0.26256513699781475
Wikipedia augmented: 0.34995654063450676
